In [1]:
import os, certifi
os.environ["SSL_CERT_FILE"] = certifi.where()
!pip install -U sentence-transformers qdrant-client pandas pyarrow fastapi
!pip install "httpx<0.28" "pandas<3.0" --upgrade
!pip uninstall -y google-genai
!pip install "httpx<0.28" --upgrade


  Using cached pandas-3.0.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (79 kB)
Using cached pandas-3.0.0-cp313-cp313-macosx_11_0_arm64.whl (9.9 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.3
    Uninstalling pandas-2.3.3:
      Successfully uninstalled pandas-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lseg-data 2.1.1 requires pandas<3.0,>=2.0, but you have pandas 3.0.0 which is incompatible.
  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 3.0.0
    Uninstalling pandas-3.0.0:
      Successfully uninstalled pandas-3.0.0


In [2]:
# Qdrant (Docker / Server mode) config
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333

COLLECTION_NAME = "banking77"
EMBEDDING_MODEL = "intfloat/multilingual-e5-small"      

VECTOR_SIZE = 384
TOP_K = 3


In [3]:
from sentence_transformers import SentenceTransformer

class EmbeddingModel:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def encode(self, texts, is_query=False):
        if isinstance(texts, str):
            texts = [texts]

        prefix = "query: " if is_query else "passage: "
        texts = [prefix + t for t in texts]

        return self.model.encode(
            texts,
            convert_to_numpy=True,
            normalize_embeddings=True
        )


In [4]:
import pandas as pd

url = "https://huggingface.co/datasets/PolyAI/banking77/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"
df = pd.read_parquet(url)

df.head(), len(df)


(                                                text  label
 0                     I am still waiting on my card?     11
 1  What can I do if my card still hasn't arrived ...     11
 2  I have been waiting over a week. Is the card s...     11
 3  Can I track my card while it is in the process...     11
 4  How do I know if I will get my card, or if it ...     11,
 10003)

In [5]:
import os
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

print("Imported QdrantClient:", QdrantClient)


Imported QdrantClient: <class 'qdrant_client.qdrant_client.QdrantClient'>


In [8]:
from qdrant_client import QdrantClient

# Create Qdrant client
client = QdrantClient(
    host=QDRANT_HOST,
    port=QDRANT_PORT
)

print(f" Connected to Qdrant at {QDRANT_HOST}:{QDRANT_PORT}")

 Connected to Qdrant at localhost:6333


In [9]:
from qdrant_client.models import VectorParams, Distance

# Delete old collection if it exists
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)
    print("🗑️ Deleted existing collection")

# Create collection with correct vector size
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE,   # 384
        distance=Distance.COSINE
    )
)

print(f" Collection '{COLLECTION_NAME}' created with VECTOR_SIZE = {VECTOR_SIZE}")

🗑️ Deleted existing collection
 Collection 'banking77' created with VECTOR_SIZE = 384


In [10]:
from qdrant_client.models import PointStruct
from tqdm import tqdm

embedder = EmbeddingModel(EMBEDDING_MODEL)

texts = df["text"].tolist()
labels = df["label"].tolist()

embeddings = embedder.encode(texts)

BATCH_SIZE = 256  # safe default (you can go up to ~512)

for start in tqdm(range(0, len(texts), BATCH_SIZE)):
    end = start + BATCH_SIZE

    batch_points = [
        PointStruct(
            id=i,
            vector=embeddings[i].tolist(),
            payload={
                "text": texts[i],
                "label": int(labels[i])
            }
        )
        for i in range(start, min(end, len(texts)))
    ]

    client.upsert(
        collection_name=COLLECTION_NAME,
        points=batch_points
    )

print(" Data ingested into Qdrant (batched)")


100%|██████████| 40/40 [00:02<00:00, 13.52it/s]

 Data ingested into Qdrant (batched)


In [11]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

def semantic_search(query, top_k=TOP_K, label_filter=None):
    # Encode query
    query_vector = embedder.encode(query, is_query=True)[0]

    # Optional label filter
    q_filter = None
    if label_filter is not None:
        q_filter = Filter(
            must=[
                FieldCondition(
                    key="label",
                    match=MatchValue(value=int(label_filter))
                )
            ]
        )

    # Query Qdrant
    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector.tolist(),
        limit=top_k,
        query_filter=q_filter,
        with_payload=True
    )

    # Server mode returns ScoredPoint list
    return list(results.points)


In [12]:
def calculate_confidence(results):
    if not results:
        return 0.0

    scores = [r.score for r in results]

    if len(scores) == 1:
        return round(scores[0], 3)

    top, second = scores[0], scores[1]
    relative_gap = max(top - second, 0.0)
    normalized = top / max(sum(scores), 1e-6)

    return round((0.7 * normalized) + (0.3 * relative_gap), 3)


In [13]:
def decide_workflow(confidence):
    if confidence >= 0.75:
        return "AUTO_EXECUTE_INTENT"
    elif confidence >= 0.50:
        return "ASK_CLARIFICATION"
    else:
        return "FALLBACK_TO_HUMAN"


In [14]:
query = "I lost my debit card"

results = semantic_search(query)

confidence = calculate_confidence(results)
workflow = decide_workflow(confidence)

response = {
    "query": query,
    "predicted_label": results[0].payload["label"] if results else None,
    "confidence": confidence,
    "workflow": workflow,
    "top_matches": [
        {
            "text": r.payload.get("text"),
            "label": r.payload.get("label"),
            "score": round(r.score, 3)
        }
        for r in results
    ]
}

response


{'query': 'I lost my debit card',
 'predicted_label': 41,
 'confidence': 0.236,
 'workflow': 'FALLBACK_TO_HUMAN',
 'top_matches': [{'text': 'help, lost my card', 'label': 41, 'score': 0.908},
  {'text': 'lost my card in atm', 'label': 18, 'score': 0.902},
  {'text': 'Help! I lost my card!', 'label': 41, 'score': 0.9}]}

In [15]:
queries = [
    "I want to change my card pin", 
    "I want to know my interest rate", 
    "How much cash can I deposit in one go?",
    "Where is the nearest branch?"
]

for q in queries:
    results = semantic_search(q)
    print("\nQuery:", q)
    print("Top label:", results[0].payload["label"])
    print("Top score:", round(results[0].score, 3))



Query: I want to change my card pin
Top label: 21
Top score: 0.906

Query: I want to know my interest rate
Top label: 32
Top score: 0.892

Query: How much cash can I deposit in one go?
Top label: 58
Top score: 0.899

Query: Where is the nearest branch?
Top label: 3
Top score: 0.825


In [16]:
queries = [
    "मैंने अपना डेबिट कार्ड खो दिया है", 
    "ATM se paisa nahi nikla but amount debit ho gaya", 
    "net banking login nahi ho raha",
    "UPI पेमेंट फेल हो गया", 
    "cash withdrawal ke liye exchange rate galat hai"
]

for q in queries:
    results = semantic_search(q)
    print("\nQuery:", q)
    print("Top label:", results[0].payload["label"])
    print("Top score:", round(results[0].score, 3))



Query: मैंने अपना डेबिट कार्ड खो दिया है
Top label: 41
Top score: 0.939

Query: ATM se paisa nahi nikla but amount debit ho gaya
Top label: 20
Top score: 0.889

Query: net banking login nahi ho raha
Top label: 44
Top score: 0.872

Query: UPI पेमेंट फेल हो गया
Top label: 53
Top score: 0.88

Query: cash withdrawal ke liye exchange rate galat hai
Top label: 76
Top score: 0.87


In [17]:
queries = [
    "I want to change my card pin", 
    "I want to know my interest rate", 
    "How much cash can I deposit in one go?",
    "Where is the nearest branch?"
]

for q in queries:
    results = semantic_search(q)

    print("\n" + "="*80)
    print("Query:", q)

    # Top prediction (same as before)
    print("Predicted label:", results[0].payload["label"])
    print("Top score:", round(results[0].score, 3))

    # NEW: Top-3 semantic matches
    print("\nTop 3 semantic search results:")
    for i, r in enumerate(results[:3], start=1):
        print(
            f"{i}. Label: {r.payload['label']} | "
            f"Score: {round(r.score, 3)} | "
            f"Text: {r.payload['text']}"
        )



Query: I want to change my card pin
Predicted label: 21
Top score: 0.906

Top 3 semantic search results:
1. Label: 21 | Score: 0.906 | Text: Can I change my card PIN?
2. Label: 21 | Score: 0.903 | Text: How do I change my card PIN?
3. Label: 38 | Score: 0.901 | Text: how do i get a pin for my card

Query: I want to know my interest rate
Predicted label: 32
Top score: 0.892

Top 3 semantic search results:
1. Label: 32 | Score: 0.892 | Text: i need to know about exchange rates
2. Label: 32 | Score: 0.892 | Text: Will I know which exchange rate I will get?
3. Label: 32 | Score: 0.888 | Text: I want to know your exchange rates.

Query: How much cash can I deposit in one go?
Predicted label: 58
Top score: 0.899

Top 3 semantic search results:
1. Label: 58 | Score: 0.899 | Text: How do I deposit cash?
2. Label: 58 | Score: 0.889 | Text: how do i get top up with cash deposit
3. Label: 58 | Score: 0.889 | Text: Where can I deposit cash to top up?

Query: Where is the nearest branch?
Predicted

In [18]:
queries = [
    "card ka pin kaise change karte hain?", 
    "card ka pin change kar sakte hain?",
    "mera interest rate kitna hai?", 
    "ek baari mein kitna cash deposit ho sakta hai?",
    "mere sabse paas wali branch kaunsi hai?"
]

for q in queries:
    results = semantic_search(q)

    print("\n" + "="*80)
    print("Query:", q)

    # Top prediction (same as before)
    print("Predicted label:", results[0].payload["label"])
    print("Top score:", round(results[0].score, 3))

    # NEW: Top-3 semantic matches
    print("\nTop 3 semantic search results:")
    for i, r in enumerate(results[:3], start=1):
        print(
            f"{i}. Label: {r.payload['label']} | "
            f"Score: {round(r.score, 3)} | "
            f"Text: {r.payload['text']}"
        )



Query: card ka pin kaise change karte hain?
Predicted label: 21
Top score: 0.926

Top 3 semantic search results:
1. Label: 21 | Score: 0.926 | Text: How do I change my card PIN?
2. Label: 21 | Score: 0.922 | Text: Can I change my card PIN?
3. Label: 21 | Score: 0.916 | Text: Do you know how to change my card PIN?

Query: card ka pin change kar sakte hain?
Predicted label: 21
Top score: 0.931

Top 3 semantic search results:
1. Label: 21 | Score: 0.931 | Text: Can I change my card PIN?
2. Label: 21 | Score: 0.928 | Text: How do I change my card PIN?
3. Label: 21 | Score: 0.925 | Text: Do you know how to change my card PIN?

Query: mera interest rate kitna hai?
Predicted label: 32
Top score: 0.833

Top 3 semantic search results:
1. Label: 32 | Score: 0.833 | Text: What exchange rate is it?
2. Label: 67 | Score: 0.831 | Text: what is the average wait for transfers
3. Label: 32 | Score: 0.831 | Text: explain the interbank exchange rate

Query: ek baari mein kitna cash deposit ho sakta hai?

In [19]:
queries = [
    "मैं अपना कार्ड पिन कैसे बदल सकता हूँ?",
    "क्या मैं अपने कार्ड का पिन बदल सकता हूँ?",
    "मेरा ब्याज दर कितना है?",
    "एक बार में कितना नकद जमा किया जा सकता है?",
    "मेरे सबसे नज़दीक वाली शाखा कौन सी है?"
]


for q in queries:
    results = semantic_search(q)

    print("\n" + "="*80)
    print("Query:", q)

    # Top prediction (same as before)
    print("Predicted label:", results[0].payload["label"])
    print("Top score:", round(results[0].score, 3))

    # NEW: Top-3 semantic matches
    print("\nTop 3 semantic search results:")
    for i, r in enumerate(results[:3], start=1):
        print(
            f"{i}. Label: {r.payload['label']} | "
            f"Score: {round(r.score, 3)} | "
            f"Text: {r.payload['text']}"
        )



Query: मैं अपना कार्ड पिन कैसे बदल सकता हूँ?
Predicted label: 21
Top score: 0.944

Top 3 semantic search results:
1. Label: 21 | Score: 0.944 | Text: How do I change my card PIN?
2. Label: 21 | Score: 0.936 | Text: Do you know how to change my card PIN?
3. Label: 21 | Score: 0.934 | Text: Can I change my card PIN?

Query: क्या मैं अपने कार्ड का पिन बदल सकता हूँ?
Predicted label: 21
Top score: 0.943

Top 3 semantic search results:
1. Label: 21 | Score: 0.943 | Text: Can I change my card PIN?
2. Label: 21 | Score: 0.933 | Text: Do you know how to change my card PIN?
3. Label: 21 | Score: 0.933 | Text: Is it possible to change my PIN?

Query: मेरा ब्याज दर कितना है?
Predicted label: 56
Top score: 0.884

Top 3 semantic search results:
1. Label: 56 | Score: 0.884 | Text: What is the fee to top-up my account
2. Label: 32 | Score: 0.878 | Text: What will my exchange rate be?
3. Label: 32 | Score: 0.878 | Text: How much are the rates of exchange?

Query: एक बार में कितना नकद जमा किया जा सकता 